***GENERATED CODE FOR widgetstest PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv("hdfs://34.83.203.43:54310"+eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '1', 'mean': '', 'stddev': '', 'min': 'MI', 'max': 'MI', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Phone', 'transformation_label': 'String Indexer'}], 'feature': 'Phone', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1', 'mean': '', 'stddev': '', 'min': '363-5450', 'max': '363-5450', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Phone'}, {'feature_label': 'Phone', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Phone')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intl_Plan', 'transformation_label': 'String Indexer'}], 'feature': 'Intl_Plan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'no', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intl_Plan'}, {'feature_label': 'Intl_Plan', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Intl_Plan')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'VMail_Plan', 'transformation_label': 'String Indexer'}], 'feature': 'VMail_Plan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'no', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'VMail_Plan'}, {'feature_label': 'VMail_Plan', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('VMail_Plan')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'cluster_labels', 'transformation_label': 'String Indexer'}], 'feature': 'cluster_labels', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1', 'mean': '', 'stddev': '', 'min': 'day_callers', 'max': 'day_callers', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'cluster_labels'}, {'feature_label': 'cluster_labels', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('cluster_labels')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
#%run widgetstestHooks.ipynb
try:
	#sourcePreExecutionHook()

	predcitivechurntest = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/PredcitiveChurn Test.csv', 'filename': 'PredcitiveChurn Test.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(predcitivechurntest)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run widgetstestHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(predcitivechurntest,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1", "mean": "", "stddev": "", "min": "MI", "max": "MI", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Account_Length", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "65.0", "stddev": "nan", "min": "65", "max": "65", "missing": "0"}, "updatedLabel": "Account_Length"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Area_Code", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "415.0", "stddev": "nan", "min": "415", "max": "415", "missing": "0"}, "updatedLabel": "Area_Code"}, {"transformationsData": [{"feature_label": "Phone", "transformation_label": "String Indexer"}], "feature": "Phone", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1", "mean": "", "stddev": "", "min": "363-5450", "max": "363-5450", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Phone"}, {"transformationsData": [{"feature_label": "Intl_Plan", "transformation_label": "String Indexer"}], "feature": "Intl_Plan", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1", "mean": "", "stddev": "", "min": "no", "max": "no", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intl_Plan"}, {"transformationsData": [{"feature_label": "VMail_Plan", "transformation_label": "String Indexer"}], "feature": "VMail_Plan", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1", "mean": "", "stddev": "", "min": "no", "max": "no", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "VMail_Plan"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "VMail_Message", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "0.0", "stddev": "nan", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "VMail_Message"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Day_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "211.3", "stddev": "nan", "min": "211.3", "max": "211.3", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Day_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Day_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "120.0", "stddev": "nan", "min": "120", "max": "120", "missing": "0"}, "updatedLabel": "Day_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Day_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "35.92", "stddev": "nan", "min": "35.92", "max": "35.92", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Day_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Eve_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "162.6", "stddev": "nan", "min": "162.6", "max": "162.6", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Eve_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Eve_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "122.0", "stddev": "nan", "min": "122", "max": "122", "missing": "0"}, "updatedLabel": "Eve_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Eve_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "13.82", "stddev": "nan", "min": "13.82", "max": "13.82", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Eve_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Night_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "134.7", "stddev": "nan", "min": "134.7", "max": "134.7", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Night_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Night_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "118.0", "stddev": "nan", "min": "118", "max": "118", "missing": "0"}, "updatedLabel": "Night_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Night_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "6.06", "stddev": "nan", "min": "6.06", "max": "6.06", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Night_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intl_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "13.2", "stddev": "nan", "min": "13.2", "max": "13.2", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intl_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "521.8", "stddev": "nan", "min": "521.8", "max": "521.8", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intl_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "5.0", "stddev": "nan", "min": "5", "max": "5", "missing": "0"}, "updatedLabel": "Intl_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intl_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "3.56", "stddev": "nan", "min": "3.56", "max": "3.56", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intl_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1", "mean": "59.36", "stddev": "nan", "min": "59.36", "max": "59.36", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Total_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustServ_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "3.0", "stddev": "nan", "min": "3", "max": "3", "missing": "0"}, "updatedLabel": "CustServ_Calls"}, {"transformationsData": [{"feature_label": "cluster_labels", "transformation_label": "String Indexer"}], "feature": "cluster_labels", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1", "mean": "", "stddev": "", "min": "day_callers", "max": "day_callers", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "cluster_labels"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Churn", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "0.0", "stddev": "nan", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "Churn"}, {"feature": "State_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1", "mean": "0.0", "stddev": "nan", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "State_stringindexer_trans..."}, {"feature": "Phone_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1", "mean": "0.0", "stddev": "nan", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "Phone_stringindexer_trans..."}, {"feature": "Intl_Plan_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1", "mean": "0.0", "stddev": "nan", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "Intl_Plan_stringindexer_t..."}, {"feature": "VMail_Plan_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1", "mean": "0.0", "stddev": "nan", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "VMail_Plan_stringindexer_..."}, {"feature": "cluster_labels_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1", "mean": "0.0", "stddev": "nan", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "cluster_labels_stringinde..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run widgetstestHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(autofe, ["State_stringindexer", "Account_Length", "Area_Code", "Phone_stringindexer", "Intl_Plan_stringindexer", "VMail_Plan_stringindexer", "VMail_Message", "Day_Mins", "Day_Calls", "Day_Charge", "Eve_Mins", "Eve_Calls", "Eve_Charge", "Night_Mins", "Night_Calls", "Night_Charge", "Intl_Mins", "total_Mins", "Intl_Calls", "Intl_Charge", "Total_Charge", "CustServ_Calls", "cluster_labels_stringindexer"], "Churn")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

